In [1]:
# Calling libraries:
from __future__ import division
%matplotlib inline
import numpy as np, time, matplotlib.pyplot as plt, math, pandas, numpy.random as npr, multiprocessing as mp, torch, copy
from pylab import plot, show, legend
from time import time
from scipy.stats import *
from tqdm import trange
from ecology_functions import *

In [2]:
T = 100
I = 5  # number of locations
J = 3  # number of species
K = 2   # number of latent factors

In [3]:
lmbda = torch.randn(J,K, requires_grad=True)
alpha = torch.randn(J,1, requires_grad=True)
c = torch.tensor(0., requires_grad=True)
phi = torch.tensor(0.5, requires_grad=True)
logsigmasq = torch.tensor(0., requires_grad=True)
x_0 = torch.randn(I,K, requires_grad=False)

theta = [alpha, lmbda, c, phi, logsigmasq]
theta_np = [theta[i].detach().numpy() for i in range(5)]
Y, X = simulate_data(x_0, T, J, theta)

In [4]:
n_particles = 1_000
rep = 20
alpha_grad = np.zeros((rep,J,1))
lmbda_grad = np.zeros((rep,J,K))
c_grad = np.zeros(rep)
phi_grad = np.zeros(rep)
logsigmasq_grad = np.zeros(rep)
for r in trange(rep) :
    theta = [copy.deepcopy(alpha), copy.deepcopy(lmbda), copy.deepcopy(c), copy.deepcopy(phi), copy.deepcopy(logsigmasq)]
    logNC = bootstrap_PF_grad_autodiff(x_0, n_particles, theta, Y)
    alpha_grad[r] = theta[0].grad.detach()
    lmbda_grad[r] = theta[1].grad.detach()
    c_grad[r] = theta[2].grad.detach()
    phi_grad[r] = theta[3].grad.detach()
    logsigmasq_grad[r] = theta[4].grad.detach()

100%|██████████| 20/20 [00:15<00:00,  1.26it/s]


In [5]:
print(np.mean(alpha_grad,0))
print(np.mean(lmbda_grad,0))
print(np.mean(c_grad,0))
print(np.mean(phi_grad,0))
print(np.mean(logsigmasq_grad,0))

[[-212.93578262]
 [-126.33988342]
 [-176.87940903]]
[[-30.80315576  15.57667055]
 [ 15.99725367  -5.02096578]
 [-16.54748445  -1.43537828]]
18.684779930114747
67.58846230506897
44.177459478378296


In [14]:
tau = np.asarray([1e-4, 1e-3, 1e-4, 1e-3, 1e-3])/10**3

In [17]:
n_particles = 1000
n_mcmc = 100
theta_chain, accept_probs = pMCMC_mala(x_0, Y, theta, n_particles, n_mcmc, tau, power=1)

  0%|          | 0/100 [00:00<?, ?it/s]/home/postdoc/dsen/Desktop/codes/DC-BATS/ecology/ecology_functions.py:396: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  a = power*(ll_proposed-torch.tensor(ll_current)) + (log_prior_proposed-log_prior_current)
 63%|██████▎   | 63/100 [00:51<00:31,  1.17it/s]

RuntimeError: assigned grad has data of a different size

In [ ]:
accepted

In [59]:
a = torch.tensor(1.2, requires_grad=True)
res = a*torch.ones(10)
# res.backward()

In [62]:
res[1].backward()

In [63]:
a

tensor(1.2000, requires_grad=True)

In [12]:
print(np.var(alpha_grad,0))
print(np.var(lmbda_grad,0))
print(np.var(c_grad))
print(np.var(phi_grad))
print(np.var(logsigmasq_grad))

[[1.34343281e-09]
 [1.32862478e-09]
 [1.41325872e-09]]
[[425.32674606 488.72661034]
 [447.27663419 514.1390499 ]
 [430.82065309 475.72450216]]
6.011454388499261e-10
1730.4517527633598
121.51384155282521


In [13]:
n_particles = 10_000
rep = 100
alpha_grad = np.zeros((rep,J,1))
lmbda_grad = np.zeros((rep,J,K))
c_grad = np.zeros(rep)
phi_grad = np.zeros(rep)
logsigmasq_grad = np.zeros(rep)
for r in trange(rep) :
    theta = [copy.deepcopy(alpha), copy.deepcopy(lmbda), copy.deepcopy(c), copy.deepcopy(phi), copy.deepcopy(logsigmasq)]
    logNC = bootstrap_PF_grad_autodiff(x_0, n_particles, theta, Y)
    alpha_grad[r] = theta[0].grad.detach()
    lmbda_grad[r] = theta[1].grad.detach()
    c_grad[r] = theta[2].grad.detach()
    phi_grad[r] = theta[3].grad.detach()
    logsigmasq_grad[r] = theta[4].grad.detach()

100%|██████████| 100/100 [08:02<00:00,  4.82s/it]


In [14]:
print(np.var(alpha_grad,0))
print(np.var(lmbda_grad,0))
print(np.var(c_grad))
print(np.var(phi_grad))
print(np.var(logsigmasq_grad))

[[3.22087668e-08]
 [2.48453347e-08]
 [3.16728605e-08]]
[[356.61384121 514.57233179]
 [454.6916215  509.74419351]
 [354.31965796 459.24901315]]
2.4879351258277895e-09
2053.3949930821072
165.29161700219717


In [20]:
a = torch.sum(alpha**2)
a.backward()

In [21]:
tau = 1e-3
alpha + tau*alpha.grad.detach() + np.sqrt(2*tau)*torch.randn(*np.shape(alpha))

tensor([[ 1.7364],
        [-0.3087],
        [ 0.3533]], grad_fn=<AddBackward0>)

In [29]:
alpha.grad = torch.zeros(*np.shape(alpha))

In [28]:
alpha.grad

tensor([[0.],
        [0.],
        [0.]])